In [ ]:
# @title Install Dependencies (Colab-Compatible)
# TA-Lib requires system-level installation on Colab
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr > /dev/null 2>&1
!make > /dev/null 2>&1
!make install > /dev/null 2>&1
%cd ..
!rm -rf ta-lib ta-lib-0.4.0-src.tar.gz

# Now install Python packages
!pip install -q yfinance pyts deap gymnasium TA-Lib

print("✓ All dependencies installed!")

In [ ]:
# @title Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
import torch
import yfinance as yf
import talib
from pyts.image import GramianAngularField
from deap import base, creator, tools, gp, algorithms
import operator
import gymnasium as gym
from gymnasium import spaces
import warnings

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
print("✓ All libraries imported!")

In [ ]:
# @title Stage 1: ULTIMATE Data Preparation (50+ Indicators)
# 1. Get data (10 years minimum for robust patterns)
print("Loading data...")
ticker = "SPY"  # or your asset
df = yf.download(ticker, start="2014-01-01", end="2024-12-31", progress=False)

# Handle MultiIndex columns if present (yfinance update)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

print(f"✓ Loaded {len(df)} days of data")

# 2. ULTIMATE Feature Engineering - 50+ Indicators
if len(df) > 200:
    close = df['Close'].values.astype(float)
    high = df['High'].values.astype(float)
    low = df['Low'].values.astype(float)
    open_price = df['Open'].values.astype(float)
    volume = df['Volume'].values.astype(float)
    
    # =============================================
    # === EMA RIBBON (8 EMAs for trend clarity) ===
    # =============================================
    df['EMA8'] = talib.EMA(close, timeperiod=8)
    df['EMA13'] = talib.EMA(close, timeperiod=13)
    df['EMA21'] = talib.EMA(close, timeperiod=21)
    df['EMA34'] = talib.EMA(close, timeperiod=34)
    df['EMA55'] = talib.EMA(close, timeperiod=55)
    df['EMA89'] = talib.EMA(close, timeperiod=89)
    df['EMA144'] = talib.EMA(close, timeperiod=144)
    df['EMA200'] = talib.EMA(close, timeperiod=200)
    
    # EMA Ribbon Spread (compression = breakout coming)
    df['EMA_Ribbon_Width'] = (df['EMA8'] - df['EMA200']) / df['Close'] * 100
    df['EMA_Fast_Slow_Ratio'] = df['EMA8'] / df['EMA55']
    df['EMA_Trend_Strength'] = (df['EMA8'] > df['EMA21']).astype(int) + \
                               (df['EMA21'] > df['EMA34']).astype(int) + \
                               (df['EMA34'] > df['EMA55']).astype(int) + \
                               (df['EMA55'] > df['EMA89']).astype(int)
    
    # =============================================
    # === MOVING AVERAGES ===
    # =============================================
    df['SMA5'] = talib.SMA(close, timeperiod=5)
    df['SMA10'] = talib.SMA(close, timeperiod=10)
    df['SMA20'] = talib.SMA(close, timeperiod=20)
    df['SMA50'] = talib.SMA(close, timeperiod=50)
    df['SMA100'] = talib.SMA(close, timeperiod=100)
    df['SMA200'] = talib.SMA(close, timeperiod=200)
    
    # Price vs MAs (where price sits relative to key levels)
    df['Price_vs_SMA20'] = (df['Close'] - df['SMA20']) / df['SMA20'] * 100
    df['Price_vs_SMA50'] = (df['Close'] - df['SMA50']) / df['SMA50'] * 100
    df['Price_vs_SMA200'] = (df['Close'] - df['SMA200']) / df['SMA200'] * 100
    
    # =============================================
    # === MOMENTUM INDICATORS ===
    # =============================================
    df['RSI'] = talib.RSI(close, timeperiod=14)
    df['RSI_Fast'] = talib.RSI(close, timeperiod=7)
    df['RSI_Slow'] = talib.RSI(close, timeperiod=21)
    df['RSI_Divergence'] = df['RSI_Fast'] - df['RSI_Slow']
    
    # Stochastic
    df['STOCH_K'], df['STOCH_D'] = talib.STOCH(high, low, close, 
                                               fastk_period=14, slowk_period=3, slowd_period=3)
    df['STOCH_Cross'] = df['STOCH_K'] - df['STOCH_D']
    
    # Williams %R
    df['WILLR'] = talib.WILLR(high, low, close, timeperiod=14)
    
    # CCI - Commodity Channel Index
    df['CCI'] = talib.CCI(high, low, close, timeperiod=20)
    df['CCI_Fast'] = talib.CCI(high, low, close, timeperiod=10)
    
    # Ultimate Oscillator
    df['ULTOSC'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    
    # ROC - Rate of Change
    df['ROC'] = talib.ROC(close, timeperiod=10)
    df['ROC_Fast'] = talib.ROC(close, timeperiod=5)
    df['ROC_Slow'] = talib.ROC(close, timeperiod=20)
    
    # Momentum
    df['MOM'] = talib.MOM(close, timeperiod=10)
    
    # =============================================
    # === MACD (Multiple Timeframes) ===
    # =============================================
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(close, 
                                                                 fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD_Fast'], df['MACD_Fast_Signal'], df['MACD_Fast_Hist'] = talib.MACD(close, 
                                                                                fastperiod=8, slowperiod=17, signalperiod=9)
    
    # =============================================
    # === VOLATILITY INDICATORS ===
    # =============================================
    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    df['ATR_Fast'] = talib.ATR(high, low, close, timeperiod=7)
    df['ATR_Percent'] = df['ATR'] / df['Close'] * 100
    
    # Bollinger Bands
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = talib.BBANDS(close, timeperiod=20)
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle'] * 100
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'] + 1e-8)
    
    # Keltner Channels (approximation)
    df['KC_Middle'] = df['EMA21']
    df['KC_Upper'] = df['EMA21'] + 2 * df['ATR']
    df['KC_Lower'] = df['EMA21'] - 2 * df['ATR']
    df['KC_Position'] = (df['Close'] - df['KC_Lower']) / (df['KC_Upper'] - df['KC_Lower'] + 1e-8)
    
    # Squeeze (BB inside KC = compression)
    df['Squeeze'] = ((df['BB_Lower'] > df['KC_Lower']) & (df['BB_Upper'] < df['KC_Upper'])).astype(int)
    
    # Historical Volatility
    df['HV_10'] = df['Close'].pct_change().rolling(10).std() * np.sqrt(252) * 100
    df['HV_20'] = df['Close'].pct_change().rolling(20).std() * np.sqrt(252) * 100
    df['HV_Ratio'] = df['HV_10'] / (df['HV_20'] + 1e-8)
    
    # =============================================
    # === TREND INDICATORS ===
    # =============================================
    # ADX - Trend Strength
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['PLUS_DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['MINUS_DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['DI_Spread'] = df['PLUS_DI'] - df['MINUS_DI']
    
    # Aroon
    df['AROON_Up'], df['AROON_Down'] = talib.AROON(high, low, timeperiod=14)
    df['AROON_Osc'] = df['AROON_Up'] - df['AROON_Down']
    
    # Parabolic SAR
    df['SAR'] = talib.SAR(high, low, acceleration=0.02, maximum=0.2)
    df['SAR_Signal'] = np.where(df['Close'] > df['SAR'], 1, -1)
    
    # =============================================
    # === VOLUME INDICATORS ===
    # =============================================
    df['Volume_MA'] = df['Volume'].rolling(20).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']
    
    # OBV - On Balance Volume
    df['OBV'] = talib.OBV(close, volume)
    df['OBV_EMA'] = talib.EMA(df['OBV'].values, timeperiod=20)
    df['OBV_Trend'] = df['OBV'] - df['OBV_EMA']
    
    # AD - Accumulation/Distribution
    df['AD'] = talib.AD(high, low, close, volume)
    df['AD_EMA'] = talib.EMA(df['AD'].values, timeperiod=20)
    
    # MFI - Money Flow Index
    df['MFI'] = talib.MFI(high, low, close, volume, timeperiod=14)
    
    # VWAP approximation (rolling)
    df['VWAP'] = (df['Close'] * df['Volume']).rolling(20).sum() / df['Volume'].rolling(20).sum()
    df['Price_vs_VWAP'] = (df['Close'] - df['VWAP']) / df['VWAP'] * 100
    
    # =============================================
    # === CANDLESTICK PATTERNS ===
    # =============================================
    df['CDL_DOJI'] = talib.CDLDOJI(open_price, high, low, close)
    df['CDL_HAMMER'] = talib.CDLHAMMER(open_price, high, low, close)
    df['CDL_ENGULFING'] = talib.CDLENGULFING(open_price, high, low, close)
    df['CDL_MORNINGSTAR'] = talib.CDLMORNINGSTAR(open_price, high, low, close)
    df['CDL_EVENINGSTAR'] = talib.CDLEVENINGSTAR(open_price, high, low, close)
    df['CDL_3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(open_price, high, low, close)
    df['CDL_3BLACKCROWS'] = talib.CDL3BLACKCROWS(open_price, high, low, close)
    df['CDL_MARUBOZU'] = talib.CDLMARUBOZU(open_price, high, low, close)
    
    # =============================================
    # === PRICE ACTION ===
    # =============================================
    df['Body_Size'] = abs(df['Close'] - df['Open']) / df['Open'] * 100
    df['Upper_Shadow'] = (df['High'] - df[['Open', 'Close']].max(axis=1)) / df['Open'] * 100
    df['Lower_Shadow'] = (df[['Open', 'Close']].min(axis=1) - df['Low']) / df['Open'] * 100
    df['Range'] = (df['High'] - df['Low']) / df['Low'] * 100
    
    # Higher Highs / Lower Lows
    df['HH'] = (df['High'] > df['High'].shift(1)).astype(int)
    df['LL'] = (df['Low'] < df['Low'].shift(1)).astype(int)
    df['HH_Count'] = df['HH'].rolling(5).sum()
    df['LL_Count'] = df['LL'].rolling(5).sum()
    
    # Gap Analysis
    df['Gap'] = (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100
    df['Gap_Up'] = (df['Gap'] > 0.5).astype(int)
    df['Gap_Down'] = (df['Gap'] < -0.5).astype(int)
    
    # =============================================
    # === MULTI-TIMEFRAME FEATURES ===
    # =============================================
    # Returns at different horizons
    df['Return_1d'] = df['Close'].pct_change(1)
    df['Return_3d'] = df['Close'].pct_change(3)
    df['Return_5d'] = df['Close'].pct_change(5)
    df['Return_10d'] = df['Close'].pct_change(10)
    df['Return_20d'] = df['Close'].pct_change(20)
    
    # Rolling max/min (support/resistance)
    df['High_20d'] = df['High'].rolling(20).max()
    df['Low_20d'] = df['Low'].rolling(20).min()
    df['Position_in_Range'] = (df['Close'] - df['Low_20d']) / (df['High_20d'] - df['Low_20d'] + 1e-8)
    
    df.dropna(inplace=True)
    
print(f"✓ ULTIMATE Indicators computed: {len([c for c in df.columns if c not in ['Open','High','Low','Close','Volume']])} features")
print(f"  Remaining rows: {len(df)}")

# List all features
feature_categories = {
    'EMA Ribbon': ['EMA8', 'EMA13', 'EMA21', 'EMA34', 'EMA55', 'EMA89', 'EMA144', 'EMA200', 
                   'EMA_Ribbon_Width', 'EMA_Fast_Slow_Ratio', 'EMA_Trend_Strength'],
    'Moving Averages': ['SMA5', 'SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200', 
                        'Price_vs_SMA20', 'Price_vs_SMA50', 'Price_vs_SMA200'],
    'Momentum': ['RSI', 'RSI_Fast', 'RSI_Slow', 'RSI_Divergence', 'STOCH_K', 'STOCH_D', 'STOCH_Cross',
                 'WILLR', 'CCI', 'CCI_Fast', 'ULTOSC', 'ROC', 'ROC_Fast', 'ROC_Slow', 'MOM'],
    'MACD': ['MACD', 'MACD_Signal', 'MACD_Hist', 'MACD_Fast', 'MACD_Fast_Signal', 'MACD_Fast_Hist'],
    'Volatility': ['ATR', 'ATR_Fast', 'ATR_Percent', 'BB_Upper', 'BB_Middle', 'BB_Lower', 
                   'BB_Width', 'BB_Position', 'KC_Position', 'Squeeze', 'HV_10', 'HV_20', 'HV_Ratio'],
    'Trend': ['ADX', 'PLUS_DI', 'MINUS_DI', 'DI_Spread', 'AROON_Up', 'AROON_Down', 'AROON_Osc', 'SAR_Signal'],
    'Volume': ['Volume_MA', 'Volume_Ratio', 'OBV_Trend', 'MFI', 'Price_vs_VWAP'],
    'Candlestick': ['CDL_DOJI', 'CDL_HAMMER', 'CDL_ENGULFING', 'CDL_MORNINGSTAR', 'CDL_EVENINGSTAR',
                    'CDL_3WHITESOLDIERS', 'CDL_3BLACKCROWS', 'CDL_MARUBOZU'],
    'Price Action': ['Body_Size', 'Upper_Shadow', 'Lower_Shadow', 'Range', 'HH_Count', 'LL_Count', 'Gap'],
    'Multi-TF': ['Return_1d', 'Return_3d', 'Return_5d', 'Return_10d', 'Return_20d', 'Position_in_Range']
}

print("\n📊 Feature Categories:")
for cat, feats in feature_categories.items():
    available = [f for f in feats if f in df.columns]
    print(f"  {cat}: {len(available)} features")

# 3. Create GASF images (visual representation)
print("\nCreating GASF images...")

close_series = df['Close'].values.astype(float)
returns = np.log(close_series[1:] / close_series[:-1])

# IMPORTANT: image_size must be <= window_size
window_size = 20
image_size = 20  # Must be <= window_size

gasf_images = []
gasf_indices = []

# pyts GASF - image_size must match or be smaller than input length
gasf = GramianAngularField(image_size=image_size, method='summation', sample_range=(-1, 1))

for i in range(len(returns) - window_size):
    window = returns[i:i+window_size]
    
    # Normalize to [-1, 1]
    window_min = window.min()
    window_max = window.max()
    if window_max - window_min > 1e-8:
        window_norm = 2 * (window - window_min) / (window_max - window_min) - 1
    else:
        window_norm = np.zeros_like(window)
    
    try:
        gasf_img = gasf.fit_transform(window_norm.reshape(1, -1))
        gasf_images.append(gasf_img[0])
        gasf_indices.append(i + window_size)
    except Exception as e:
        if len(gasf_images) == 0 and i == 0:
            print(f"  Debug - First window error: {e}")
        continue

gasf_images = np.array(gasf_images)
print(f"✓ Created {len(gasf_images)} GASF images (size: {image_size}x{image_size})")

# 4. Create labels (future returns)
df = df.reset_index(drop=True)
df['Returns'] = np.nan
df.loc[1:len(returns), 'Returns'] = returns
df['Future_5d_Return'] = df['Returns'].rolling(5).sum().shift(-5)
df['Label'] = (df['Future_5d_Return'] > 0).astype(int)

# Align data
valid_indices = [idx for idx in gasf_indices if idx < len(df)]
aligned_future_returns = df['Future_5d_Return'].iloc[valid_indices].values
aligned_gasf_images = gasf_images[:len(valid_indices)]

mask = ~np.isnan(aligned_future_returns)
aligned_gasf_images = aligned_gasf_images[mask]
aligned_future_returns = aligned_future_returns[mask]
aligned_indices = np.array(valid_indices)[mask]

print(f"✓ Data ready for discovery. Samples: {len(aligned_gasf_images)}")

if len(aligned_gasf_images) == 0:
    print("\n⚠️ WARNING: No GASF images created!")

In [ ]:
# @title Stage 2: Unsupervised Visual Discovery
# Check if data is ready
if 'aligned_gasf_images' not in dir() or len(aligned_gasf_images) == 0:
    raise RuntimeError("⚠️ Run Cell 3 (Data Preparation) first!")

class VisualPatternFinder:
    """Cluster GASF images - find recurring visual structures."""
    
    def __init__(self, n_patterns=10):
        self.n_patterns = n_patterns
        self.kmeans = None
        self.patterns = {}
    
    def discover_visual_patterns(self, gasf_images, future_returns):
        """
        Cluster images → find which clusters are profitable.
        """
        # Reshape for clustering (flatten images)
        n_samples, h, w = gasf_images.shape
        X = gasf_images.reshape(n_samples, h*w)
        
        # Cluster
        self.kmeans = KMeans(n_clusters=self.n_patterns, random_state=42, n_init=10)
        clusters = self.kmeans.fit_predict(X)
        
        # Analyze each cluster
        print("\nVISUAL PATTERN ANALYSIS")
        print("=" * 60)
        
        for cluster_id in range(self.n_patterns):
            mask = clusters == cluster_id
            cluster_returns = future_returns[mask]
            
            if len(cluster_returns) == 0:
                continue

            # Statistics
            avg_return = np.mean(cluster_returns)
            win_rate = (cluster_returns > 0).mean()
            frequency = mask.sum() / len(clusters)
            std_dev = np.std(cluster_returns) + 1e-8
            sharpe = avg_return / std_dev * np.sqrt(252/5)  # Annualized roughly
            
            print(f"\nPattern {cluster_id}:")
            print(f"  Frequency: {frequency*100:.1f}% of days")
            print(f"  Next-5d return: {avg_return*100:.2f}%")
            print(f"  Win rate: {win_rate*100:.1f}%")
            print(f"  Sharpe: {sharpe:.2f}")
            
            if avg_return > 0.005:  # Threshold for "profitable"
                print(f"  ✓ PROFITABLE PATTERN FOUND")
            
            self.patterns[cluster_id] = {
                'avg_return': avg_return,
                'win_rate': win_rate,
                'frequency': frequency,
                'sharpe': sharpe,
                'centroid': self.kmeans.cluster_centers_[cluster_id].reshape(h, w)
            }
            
    def plot_patterns(self):
        """Visualize the centroids of discovered patterns."""
        if not self.patterns:
            print("No patterns discovered yet.")
            return
            
        fig, axes = plt.subplots(2, (self.n_patterns + 1) // 2, figsize=(15, 6))
        axes = axes.flatten()
        
        for i, (cluster_id, stats) in enumerate(self.patterns.items()):
            if i < len(axes):
                axes[i].imshow(stats['centroid'], cmap='rainbow', origin='lower')
                axes[i].set_title(f"P{cluster_id}: WR {stats['win_rate']:.2f}")
                axes[i].axis('off')
        plt.tight_layout()
        plt.show()

# Discover patterns
finder = VisualPatternFinder(n_patterns=8)
finder.discover_visual_patterns(aligned_gasf_images, aligned_future_returns)
finder.plot_patterns()

print("\n✓ DISCOVERED: Which price shapes predict returns")

In [ ]:
# @title Stage 3: ULTIMATE Rare Structure Detection (Multi-Feature)
class UltimateRareStateDetector:
    """Find anomalous market structures using ALL indicators."""
    
    def __init__(self):
        self.pca = PCA(n_components=0.95)  # Keep 95% variance
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42, n_jobs=-1)
        self.rare_profiles = {}
    
    def find_rare_states(self, df, feature_cols=None):
        """Analyze: What rare states precede big moves?"""
        
        # Use ALL numeric features if not specified
        if feature_cols is None:
            exclude_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 
                           'Future_5d_Return', 'Label', 'SAR', 'OBV', 'AD', 'VWAP',
                           'High_20d', 'Low_20d', 'BB_Upper', 'BB_Lower', 'BB_Middle',
                           'KC_Upper', 'KC_Lower', 'KC_Middle']
            feature_cols = [c for c in df.columns if c not in exclude_cols and df[c].dtype in ['float64', 'int64']]
        
        print("\nRARE STATE ANALYSIS (ULTIMATE)")
        print("=" * 60)
        print(f"Analyzing {len(feature_cols)} features...")
        
        data_subset = df[feature_cols].copy()
        data_subset.dropna(inplace=True)
        
        features = data_subset.values
        
        # Standardize
        scaler = StandardScaler()
        features = scaler.fit_transform(features)
        
        # Reduce dimensions
        features_reduced = self.pca.fit_transform(features)
        print(f"PCA: {features.shape[1]} features → {features_reduced.shape[1]} components (95% variance)")
        
        # Find anomalies at different contamination levels
        for contamination in [0.02, 0.05, 0.10]:
            self.iso_forest = IsolationForest(contamination=contamination, random_state=42)
            anomalies = self.iso_forest.fit_predict(features_reduced)
            anomaly_indices = np.where(anomalies == -1)[0]
            
            rare_returns = []
            for idx in anomaly_indices:
                df_idx = data_subset.index[idx]
                if df_idx in df.index:
                    ret = df.loc[df_idx, 'Future_5d_Return']
                    if not np.isnan(ret):
                        rare_returns.append(ret)
            
            rare_returns = np.array(rare_returns)
            
            if len(rare_returns) > 0:
                avg_ret = np.mean(rare_returns)
                win_rate = (rare_returns > 0).mean()
                
                print(f"\n📍 Top {contamination*100:.0f}% Rarest States ({len(anomaly_indices)} days):")
                print(f"   Avg Return (5d): {avg_ret*100:.2f}%")
                print(f"   Win Rate: {win_rate*100:.1f}%")
                print(f"   Best: +{np.max(rare_returns)*100:.2f}%  Worst: {np.min(rare_returns)*100:.2f}%")
                
                if avg_ret > 0.01:
                    print(f"   ✓ BULLISH SIGNAL")
                elif avg_ret < -0.01:
                    print(f"   ⚠️ BEARISH SIGNAL")
                
                self.rare_profiles[contamination] = {
                    'count': len(anomaly_indices),
                    'avg_return': avg_ret,
                    'win_rate': win_rate,
                    'indices': anomaly_indices
                }
        
        # Identify WHAT makes states rare
        print("\n🔍 Feature Contribution to Rare States:")
        best_contam = max(self.rare_profiles.keys(), key=lambda x: abs(self.rare_profiles[x]['avg_return']))
        rare_idx = self.rare_profiles[best_contam]['indices']
        
        # Compare rare vs normal
        rare_features = features[rare_idx]
        normal_features = features[~np.isin(np.arange(len(features)), rare_idx)]
        
        # Find most different features
        diffs = []
        for i, col in enumerate(feature_cols):
            if i < rare_features.shape[1]:
                rare_mean = np.mean(rare_features[:, i])
                normal_mean = np.mean(normal_features[:, i])
                diff = abs(rare_mean - normal_mean)
                diffs.append((col, diff, rare_mean, normal_mean))
        
        diffs.sort(key=lambda x: x[1], reverse=True)
        
        print("   Top discriminating features (rare vs normal):")
        for col, diff, rare_m, norm_m in diffs[:10]:
            direction = "↑" if rare_m > norm_m else "↓"
            print(f"   {direction} {col}: rare={rare_m:.2f} vs normal={norm_m:.2f}")
        
        return self.rare_profiles

detector = UltimateRareStateDetector()
rare_profiles = detector.find_rare_states(df)

In [ ]:
# @title Stage 4: ULTIMATE Symbolic Regression (25 Features!)
class UltimateFormulaEvolver:
    """Genetic programming: evolve trading formulas with 25 key features."""
    
    def __init__(self):
        # Check if creator classes already exist to avoid errors on re-run
        if hasattr(creator, "FitnessMax"):
            del creator.FitnessMax
        if hasattr(creator, "Individual"):
            del creator.Individual
            
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)
        
        self.toolbox = base.Toolbox()
        
        # === 25 KEY FEATURES for formula evolution ===
        self.feature_names = [
            # Momentum (5)
            'RSI', 'RSI_Divergence', 'STOCH_K', 'CCI', 'MOM',
            # Trend (5)
            'ADX', 'DI_Spread', 'AROON_Osc', 'EMA_Trend_Strength', 'SAR_Signal',
            # Volatility (5)
            'ATR_Percent', 'BB_Position', 'BB_Width', 'Squeeze', 'HV_Ratio',
            # Volume (3)
            'Volume_Ratio', 'OBV_Trend', 'MFI',
            # Price Position (4)
            'Price_vs_SMA20', 'Price_vs_SMA200', 'Price_vs_VWAP', 'Position_in_Range',
            # Multi-TF (3)
            'Return_1d', 'Return_5d', 'Return_20d'
        ]
        
        self.n_features = len(self.feature_names)
        self.pset = gp.PrimitiveSet("MAIN", self.n_features)
        
        # Rename arguments for clarity in output
        for i, name in enumerate(self.feature_names):
            self.pset.renameArguments(**{f'ARG{i}': name})
        
        # === OPERATIONS (including advanced ones) ===
        self.pset.addPrimitive(operator.add, 2, name='add')
        self.pset.addPrimitive(operator.sub, 2, name='sub')
        self.pset.addPrimitive(operator.mul, 2, name='mul')
        
        def protected_div(x, y):
            return x / (y + 1e-8)
        self.pset.addPrimitive(protected_div, 2, name='div')
        
        def protected_sqrt(x):
            return np.sqrt(np.abs(x))
        self.pset.addPrimitive(protected_sqrt, 1, name='sqrt')
        
        def protected_log(x):
            return np.log(np.abs(x) + 1e-8)
        self.pset.addPrimitive(protected_log, 1, name='log')
        
        def protected_exp(x):
            return np.clip(np.exp(np.clip(x, -10, 10)), -1e10, 1e10)
        self.pset.addPrimitive(protected_exp, 1, name='exp')
        
        self.pset.addPrimitive(np.sin, 1, name='sin')
        self.pset.addPrimitive(np.cos, 1, name='cos')
        self.pset.addPrimitive(np.tanh, 1, name='tanh')
        self.pset.addPrimitive(np.abs, 1, name='abs')
        self.pset.addPrimitive(operator.neg, 1, name='neg')
        
        # Max/Min for combining signals
        def safe_max(x, y):
            return np.maximum(x, y)
        def safe_min(x, y):
            return np.minimum(x, y)
        self.pset.addPrimitive(safe_max, 2, name='max')
        self.pset.addPrimitive(safe_min, 2, name='min')
        
        # Conditional (if-then-else approximation)
        def if_positive(condition, if_true, if_false):
            return np.where(condition > 0, if_true, if_false)
        self.pset.addPrimitive(if_positive, 3, name='ifpos')
        
        # Terminal names (no dots allowed)
        self.pset.addTerminal(0.0, name='zero')
        self.pset.addTerminal(0.5, name='half')
        self.pset.addTerminal(1.0, name='one')
        self.pset.addTerminal(2.0, name='two')
        self.pset.addTerminal(-1.0, name='neg1')
        self.pset.addTerminal(30.0, name='overbought')  # RSI threshold
        self.pset.addTerminal(70.0, name='oversold')     # RSI threshold
        self.pset.addTerminal(25.0, name='adx_trend')    # ADX threshold
        
        self.toolbox.register("expr", gp.genHalfAndHalf, pset=self.pset, min_=2, max_=5)
        self.toolbox.register("individual", tools.initIterate, creator.Individual, self.toolbox.expr)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        self.toolbox.register("evaluate", self.eval_formula)
        self.toolbox.register("mate", gp.cxOnePoint)
        self.toolbox.register("mutate", gp.mutUniform, expr=self.toolbox.expr, pset=self.pset)
        self.toolbox.register("select", tools.selTournament, tournsize=5)  # Increased selection pressure
        
        # Limit tree height to prevent bloat
        self.toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))
        self.toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))
    
    def eval_formula(self, individual, X, y):
        """Evaluate formula with multiple metrics."""
        try:
            func = gp.compile(individual, self.pset)
            # Unpack all 25 features
            pred = func(*[X[:,i] for i in range(self.n_features)])
            
            if np.isnan(pred).any() or np.isinf(pred).any():
                return (-100.0,)
            
            # Multiple fitness components
            correlation = np.corrcoef(pred.flatten(), y.flatten())[0, 1]
            if np.isnan(correlation):
                return (-100.0,)
            
            # Sharpe-like metric
            pred_sign = np.sign(pred)
            strategy_returns = pred_sign * y
            if np.std(strategy_returns) > 0:
                sharpe = np.mean(strategy_returns) / np.std(strategy_returns)
            else:
                sharpe = 0
            
            # Combined fitness: correlation + sharpe contribution
            fitness = correlation * 0.6 + np.tanh(sharpe) * 0.4
            
            return (fitness,)
            
        except Exception as e:
            return (-100.0,)
    
    def evolve(self, X, y, pop_size=200, generations=30):
        """Evolve formulas with larger population and more generations."""
        pop = self.toolbox.population(n=pop_size)
        hof = tools.HallOfFame(10)  # Keep top 10
        
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("max", np.max)
        
        self.toolbox.register("evaluate", self.eval_formula, X=X, y=y)
        
        pop, log = algorithms.eaSimple(
            pop, self.toolbox,
            cxpb=0.7, mutpb=0.3,  # Higher mutation for exploration
            ngen=generations,
            stats=stats,
            halloffame=hof,
            verbose=True
        )
        
        return hof, log
    
    def backtest_formula(self, formula, X, y, verbose=True):
        """Backtest a discovered formula."""
        func = gp.compile(formula, self.pset)
        pred = func(*[X[:,i] for i in range(self.n_features)])
        
        # Generate signals
        signals = np.sign(pred)
        returns = signals * y
        
        # Metrics
        total_return = (1 + returns).prod() - 1
        win_rate = (returns > 0).sum() / (returns != 0).sum() if (returns != 0).sum() > 0 else 0
        sharpe = np.mean(returns) / (np.std(returns) + 1e-8) * np.sqrt(252/5)
        max_dd = (np.maximum.accumulate(np.cumsum(returns)) - np.cumsum(returns)).max()
        
        if verbose:
            print(f"  Total Return: {total_return*100:.2f}%")
            print(f"  Win Rate: {win_rate*100:.1f}%")
            print(f"  Sharpe Ratio: {sharpe:.2f}")
            print(f"  Max Drawdown: {max_dd*100:.2f}%")
        
        return {
            'total_return': total_return,
            'win_rate': win_rate,
            'sharpe': sharpe,
            'max_dd': max_dd,
            'signals': signals
        }

# === Prepare data for Evolution with 25 features ===
print("\nFORMULA EVOLUTION (ULTIMATE)")
print("=" * 60)

# Feature list must match evolver
feature_cols = [
    'RSI', 'RSI_Divergence', 'STOCH_K', 'CCI', 'MOM',
    'ADX', 'DI_Spread', 'AROON_Osc', 'EMA_Trend_Strength', 'SAR_Signal',
    'ATR_Percent', 'BB_Position', 'BB_Width', 'Squeeze', 'HV_Ratio',
    'Volume_Ratio', 'OBV_Trend', 'MFI',
    'Price_vs_SMA20', 'Price_vs_SMA200', 'Price_vs_VWAP', 'Position_in_Range',
    'Return_1d', 'Return_5d', 'Return_20d'
]

# Check which features are available
available_features = [f for f in feature_cols if f in df.columns]
missing_features = [f for f in feature_cols if f not in df.columns]

if missing_features:
    print(f"⚠️ Missing features: {missing_features}")
    print("Using available features only...")
    feature_cols = available_features

print(f"Using {len(feature_cols)} features for formula evolution")

data_subset = df[feature_cols + ['Future_5d_Return']].dropna()

X_features = data_subset[feature_cols].values
y_returns = data_subset['Future_5d_Return'].values

# Normalize features (important for GP)
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

# === EVOLVE with SERIOUS parameters ===
print(f"\n🧬 Starting Evolution...")
print(f"   Population: 200")
print(f"   Generations: 30")
print(f"   Features: {len(feature_cols)}")
print()

evolver = UltimateFormulaEvolver()
best_formulas, log = evolver.evolve(X_features, y_returns, pop_size=200, generations=30)

print("\n" + "=" * 60)
print("🏆 TOP DISCOVERED FORMULAS (with backtests):")
print("=" * 60)

for i, formula in enumerate(best_formulas[:5]):
    fitness = formula.fitness.values[0]
    print(f"\n{i+1}. Fitness: {fitness:.4f}")
    print(f"   Formula: {str(formula)[:100]}...")
    
    # Backtest
    evolver.backtest_formula(formula, X_features, y_returns)

# Store for later use
best_formula = best_formulas[0] if best_formulas else None

In [ ]:
# @title Stage 5: ULTIMATE RL Environment (20 Feature State)
class UltimateTradingEnv(gym.Env):
    """RL environment with comprehensive market state."""
    
    def __init__(self, df, window=20):
        super(UltimateTradingEnv, self).__init__()
        self.df = df.reset_index(drop=True)
        self.window = window
        self.current_idx = window
        self.max_idx = len(df) - 5
        
        # Actions: 0=SHORT, 1=HOLD, 2=LONG
        self.action_space = spaces.Discrete(3)
        
        # === 20 KEY STATE FEATURES ===
        self.state_features = [
            'RSI', 'RSI_Divergence', 'STOCH_K', 'CCI',
            'ADX', 'DI_Spread', 'EMA_Trend_Strength',
            'ATR_Percent', 'BB_Position', 'Squeeze',
            'Volume_Ratio', 'MFI',
            'Price_vs_SMA20', 'Price_vs_SMA200', 'Position_in_Range',
            'Return_1d', 'Return_5d', 'MACD_Hist',
            'HH_Count', 'LL_Count'
        ]
        
        # Verify features exist
        self.state_features = [f for f in self.state_features if f in df.columns]
        
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(self.state_features),), 
            dtype=np.float32
        )
        
        # Normalize features using the whole dataset
        self.scaler = StandardScaler()
        valid_data = df[self.state_features].dropna()
        self.scaler.fit(valid_data)
        
        # Track position
        self.position = 0  # -1, 0, 1
        self.entry_price = 0
        self.pnl = 0
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_idx = np.random.randint(self.window, max(self.window + 1, self.max_idx - 200))
        self.position = 0
        self.entry_price = 0
        self.pnl = 0
        return self._get_state(), {}
    
    def step(self, action):
        current_price = self.df['Close'].iloc[self.current_idx]
        next_price = self.df['Close'].iloc[min(self.current_idx + 1, len(self.df) - 1)]
        
        # Position mapping: 0=SHORT, 1=HOLD, 2=LONG
        new_position = action - 1
        
        # Calculate reward based on actual position
        price_change = (next_price - current_price) / current_price
        reward = float(new_position * price_change * 100)  # Percentage PnL
        
        # Transaction cost
        if new_position != self.position:
            reward -= 0.01  # 0.01% transaction cost
        
        self.position = new_position
        self.current_idx += 1
        
        terminated = self.current_idx >= self.max_idx
        truncated = False
        
        return self._get_state(), reward, terminated, truncated, {'pnl': reward}
    
    def _get_state(self):
        idx = min(self.current_idx, len(self.df) - 1)
        
        state = []
        for feat in self.state_features:
            val = self.df[feat].iloc[idx]
            if np.isnan(val):
                val = 0.0
            state.append(val)
        
        state = np.array(state, dtype=np.float32).reshape(1, -1)
        state = self.scaler.transform(state).flatten()
        
        return state.astype(np.float32)

# === Simple Q-Learning Agent (no deep learning needed) ===
class SimpleQAgent:
    """Tabular Q-learning agent for market pattern discovery."""
    
    def __init__(self, n_states=100, n_actions=3, lr=0.1, gamma=0.95, epsilon=0.3):
        self.n_states = n_states
        self.n_actions = n_actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((n_states, n_actions))
        self.state_bins = None
    
    def discretize_state(self, state):
        """Convert continuous state to discrete bin."""
        if self.state_bins is None:
            return 0
        
        # Use first 3 features for simple discretization
        bins = []
        for i in range(min(3, len(state))):
            bin_idx = np.digitize(state[i], self.state_bins[i]) - 1
            bin_idx = np.clip(bin_idx, 0, 4)  # 5 bins per feature
            bins.append(bin_idx)
        
        # Combine into single state index
        state_idx = bins[0] * 25 + bins[1] * 5 + bins[2] if len(bins) >= 3 else bins[0]
        return min(state_idx, self.n_states - 1)
    
    def setup_bins(self, states):
        """Create discretization bins from data."""
        n_bins = 5
        self.state_bins = []
        for i in range(min(3, states.shape[1])):
            percentiles = np.percentile(states[:, i], np.linspace(0, 100, n_bins + 1)[1:-1])
            self.state_bins.append(percentiles)
    
    def act(self, state):
        """Epsilon-greedy action selection."""
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        state_idx = self.discretize_state(state)
        return np.argmax(self.q_table[state_idx])
    
    def update(self, state, action, reward, next_state, done):
        """Q-learning update."""
        state_idx = self.discretize_state(state)
        next_state_idx = self.discretize_state(next_state)
        
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.q_table[next_state_idx])
        
        self.q_table[state_idx, action] += self.lr * (target - self.q_table[state_idx, action])

# === Create and Train Agent ===
print("\nRL ENVIRONMENT & TRAINING")
print("=" * 60)

env = UltimateTradingEnv(df)
print(f"Observation space: {env.observation_space} ({len(env.state_features)} features)")
print(f"Action space: {env.action_space}")
print(f"State features: {env.state_features[:5]}... +{len(env.state_features)-5} more")

# Collect sample states for discretization
print("\nCollecting state samples...")
sample_states = []
for _ in range(100):
    state, _ = env.reset()
    sample_states.append(state)
sample_states = np.array(sample_states)

# Create and train agent
agent = SimpleQAgent(n_states=125, n_actions=3, lr=0.1, gamma=0.95, epsilon=0.2)
agent.setup_bins(sample_states)

print("\n🤖 Training Q-Learning Agent (500 episodes)...")
episode_rewards = []

for episode in range(500):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(100):  # Max 100 steps per episode
        action = agent.act(state)
        next_state, reward, done, truncated, info = env.step(action)
        agent.update(state, action, reward, next_state, done)
        
        state = next_state
        total_reward += reward
        
        if done or truncated:
            break
    
    episode_rewards.append(total_reward)
    
    # Decay epsilon
    agent.epsilon = max(0.05, agent.epsilon * 0.995)
    
    if (episode + 1) % 100 == 0:
        avg_reward = np.mean(episode_rewards[-100:])
        print(f"   Episode {episode+1}: Avg Reward = {avg_reward:.2f}")

# Analyze learned policy
print("\n📊 Learned Policy Analysis:")
print("Q-Table Summary (action preferences by market state):")
print("   State Region → [SHORT, HOLD, LONG]")

# Show some interesting states
for i in range(0, 125, 25):
    q_vals = agent.q_table[i]
    best_action = ['SHORT', 'HOLD', 'LONG'][np.argmax(q_vals)]
    print(f"   State {i:3d}: {q_vals.round(2)} → {best_action}")

print("\n✓ Agent trained and ready!")

In [ ]:
# @title Stage 6: ULTIMATE Multimodal Fusion
class UltimateEnsemble:
    """Combine all discovery methods with confidence weighting."""
    
    def __init__(self, visual_finder, rare_detector, best_formula, evolver, rl_agent, env):
        self.visual = visual_finder
        self.rare = rare_detector
        self.formula = best_formula
        self.evolver = evolver
        self.agent = rl_agent
        self.env = env
        
        if self.formula is not None:
            self.compiled_formula = gp.compile(self.formula, self.evolver.pset)
        else:
            self.compiled_formula = None
        
        # Track performance for confidence weighting
        self.method_accuracy = {
            'visual': 0.5,
            'genetic': 0.5,
            'rare': 0.5,
            'rl': 0.5
        }
    
    def get_ensemble_signal(self, row_dict, gasf_img=None):
        """Query all modalities and combine signals."""
        
        votes = {}
        confidences = {}
        
        # 1. Visual Pattern Vote
        if gasf_img is not None and self.visual.kmeans is not None:
            try:
                cluster = self.visual.kmeans.predict(gasf_img.reshape(1, -1))[0]
                pattern_stats = self.visual.patterns.get(cluster, {})
                avg_ret = pattern_stats.get('avg_return', 0)
                votes['visual'] = np.sign(avg_ret)
                confidences['visual'] = min(abs(avg_ret) * 100, 1.0)  # Scale confidence
            except:
                votes['visual'] = 0
                confidences['visual'] = 0
        else:
            votes['visual'] = 0
            confidences['visual'] = 0
            
        # 2. Genetic Formula Vote
        if self.compiled_formula is not None:
            try:
                # Build feature array from row_dict
                features = []
                for feat in self.evolver.feature_names:
                    val = row_dict.get(feat, 0)
                    if np.isnan(val):
                        val = 0
                    features.append(val)
                
                f_val = self.compiled_formula(*features)
                votes['genetic'] = float(np.sign(f_val))
                confidences['genetic'] = min(abs(f_val), 1.0)
            except:
                votes['genetic'] = 0
                confidences['genetic'] = 0
        else:
            votes['genetic'] = 0
            confidences['genetic'] = 0
            
        # 3. Rare State Vote
        # Check if current state is rare (use ADX + BB_Width + Volume_Ratio)
        try:
            adx = row_dict.get('ADX', 25)
            bb_width = row_dict.get('BB_Width', 10)
            vol_ratio = row_dict.get('Volume_Ratio', 1)
            
            # Rare = low ADX + narrow bands + high volume (squeeze breakout)
            is_squeeze = bb_width < 5 and vol_ratio > 1.5
            if is_squeeze:
                votes['rare'] = 1  # Squeeze = bullish bias
                confidences['rare'] = 0.7
            else:
                votes['rare'] = 0
                confidences['rare'] = 0.3
        except:
            votes['rare'] = 0
            confidences['rare'] = 0
            
        # 4. RL Agent Vote
        if self.agent is not None:
            try:
                state = []
                for feat in self.env.state_features:
                    val = row_dict.get(feat, 0)
                    if np.isnan(val):
                        val = 0
                    state.append(val)
                state = np.array(state, dtype=np.float32)
                
                action = self.agent.act(state)
                votes['rl'] = action - 1  # Convert to -1, 0, 1
                
                # Confidence from Q-value spread
                state_idx = self.agent.discretize_state(state)
                q_vals = self.agent.q_table[state_idx]
                q_spread = np.max(q_vals) - np.mean(q_vals)
                confidences['rl'] = min(q_spread / 10, 1.0)
            except:
                votes['rl'] = 0
                confidences['rl'] = 0
        else:
            votes['rl'] = 0
            confidences['rl'] = 0
        
        # Weighted consensus
        total_weight = sum(confidences.values())
        if total_weight > 0:
            weighted_signal = sum(v * confidences[k] for k, v in votes.items()) / total_weight
        else:
            weighted_signal = 0
        
        return {
            'signal': weighted_signal,
            'direction': 'LONG' if weighted_signal > 0.2 else 'SHORT' if weighted_signal < -0.2 else 'NEUTRAL',
            'votes': votes,
            'confidences': confidences,
            'consensus_strength': abs(weighted_signal)
        }
    
    def backtest_ensemble(self, df, gasf_images=None, aligned_indices=None):
        """Backtest the full ensemble on historical data."""
        
        print("\n📈 ENSEMBLE BACKTEST")
        print("=" * 60)
        
        signals = []
        returns = []
        
        # Sample positions to avoid overfitting
        test_indices = range(200, min(len(df)-5, len(df)), 5)  # Every 5th day
        
        for idx in test_indices:
            if idx >= len(df) - 5:
                continue
                
            row = df.iloc[idx].to_dict()
            
            # Get GASF if available
            gasf_img = None
            if gasf_images is not None and aligned_indices is not None:
                matching = np.where(aligned_indices == idx)[0]
                if len(matching) > 0:
                    gasf_img = gasf_images[matching[0]]
            
            result = self.get_ensemble_signal(row, gasf_img)
            signal = result['signal']
            
            # Get actual future return
            future_ret = df['Future_5d_Return'].iloc[idx]
            if np.isnan(future_ret):
                continue
            
            signals.append(signal)
            returns.append(future_ret)
        
        signals = np.array(signals)
        returns = np.array(returns)
        
        # Calculate strategy returns
        positions = np.sign(signals)
        strategy_returns = positions * returns
        
        # Metrics
        total_return = (1 + strategy_returns).prod() - 1
        buy_hold_return = (1 + returns).prod() - 1
        win_rate = (strategy_returns > 0).sum() / (strategy_returns != 0).sum() if (strategy_returns != 0).sum() > 0 else 0
        
        avg_win = strategy_returns[strategy_returns > 0].mean() if (strategy_returns > 0).sum() > 0 else 0
        avg_loss = abs(strategy_returns[strategy_returns < 0].mean()) if (strategy_returns < 0).sum() > 0 else 0
        profit_factor = avg_win / avg_loss if avg_loss > 0 else 999
        
        sharpe = np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-8) * np.sqrt(252/5)
        
        print(f"Strategy Return: {total_return*100:.2f}%")
        print(f"Buy & Hold Return: {buy_hold_return*100:.2f}%")
        print(f"Alpha Generated: {(total_return - buy_hold_return)*100:.2f}%")
        print(f"Win Rate: {win_rate*100:.1f}%")
        print(f"Profit Factor: {profit_factor:.2f}")
        print(f"Sharpe Ratio: {sharpe:.2f}")
        print(f"Trades: {(positions != 0).sum()}")
        
        return {
            'total_return': total_return,
            'buy_hold': buy_hold_return,
            'alpha': total_return - buy_hold_return,
            'win_rate': win_rate,
            'sharpe': sharpe
        }

# Create ensemble with all components
print("\n🎯 BUILDING ULTIMATE ENSEMBLE")
print("=" * 60)

ensemble = UltimateEnsemble(
    visual_finder=finder,
    rare_detector=detector,
    best_formula=best_formula,
    evolver=evolver,
    rl_agent=agent,
    env=env
)

print("✓ Ensemble integrates:")
print("  1. Visual Pattern Recognition (GASF + KMeans)")
print("  2. Genetic Formula Discovery (25 features)")
print("  3. Rare State Detection (Multi-feature anomaly)")
print("  4. RL Agent (Q-Learning policy)")

# Run backtest
backtest_results = ensemble.backtest_ensemble(df, aligned_gasf_images, aligned_indices)

In [ ]:
# @title Stage 7: ULTIMATE Summary & Export
print("\n" + "=" * 70)
print("🎯 ALPHA DISCOVERY COMPLETE - ULTIMATE EDITION")
print("=" * 70)

print("\n📊 WHAT WAS DISCOVERED:")
print("-" * 50)

# 1. Visual Patterns
print(f"\n1️⃣  VISUAL PATTERNS: {len(finder.patterns)} discovered")
best_pattern = max(finder.patterns.items(), key=lambda x: x[1]['win_rate'])
print(f"    Best: Pattern {best_pattern[0]} → {best_pattern[1]['win_rate']*100:.1f}% win rate")
print(f"    Sharpe: {best_pattern[1]['sharpe']:.2f}")

# 2. Rare States
print(f"\n2️⃣  RARE STATES:")
for contam, profile in detector.rare_profiles.items():
    print(f"    Top {contam*100:.0f}%: {profile['win_rate']*100:.1f}% WR, {profile['avg_return']*100:.2f}% avg return")

# 3. Evolved Formulas
print(f"\n3️⃣  GENETIC FORMULAS: {len(best_formulas)} evolved")
if best_formula:
    print(f"    Best fitness: {best_formula.fitness.values[0]:.4f}")
    formula_str = str(best_formula)
    if len(formula_str) > 80:
        formula_str = formula_str[:80] + "..."
    print(f"    Formula: {formula_str}")

# 4. RL Agent
print(f"\n4️⃣  RL AGENT: Trained (500 episodes)")
print(f"    State features: {len(env.state_features)}")
print(f"    Final epsilon: {agent.epsilon:.3f}")

# 5. Ensemble Performance
print(f"\n5️⃣  ENSEMBLE BACKTEST:")
print(f"    Total Return: {backtest_results['total_return']*100:.2f}%")
print(f"    Buy & Hold: {backtest_results['buy_hold']*100:.2f}%")
print(f"    Alpha: {backtest_results['alpha']*100:.2f}%")
print(f"    Win Rate: {backtest_results['win_rate']*100:.1f}%")
print(f"    Sharpe: {backtest_results['sharpe']:.2f}")

# === SAVE DISCOVERIES ===
print("\n" + "=" * 70)
print("💾 SAVING DISCOVERIES")
print("=" * 70)

import json
import pickle

# Create discoveries dict
discoveries = {
    'ticker': ticker,
    'date_range': '2014-2024',
    'visual_patterns': {
        k: {
            'avg_return': float(v['avg_return']),
            'win_rate': float(v['win_rate']),
            'frequency': float(v['frequency']),
            'sharpe': float(v['sharpe'])
        } for k, v in finder.patterns.items()
    },
    'rare_states': {
        str(k): {
            'count': int(v['count']),
            'avg_return': float(v['avg_return']),
            'win_rate': float(v['win_rate'])
        } for k, v in detector.rare_profiles.items()
    },
    'best_formula': str(best_formula) if best_formula else None,
    'best_formula_fitness': float(best_formula.fitness.values[0]) if best_formula else None,
    'ensemble_results': {
        'total_return': float(backtest_results['total_return']),
        'buy_hold': float(backtest_results['buy_hold']),
        'alpha': float(backtest_results['alpha']),
        'win_rate': float(backtest_results['win_rate']),
        'sharpe': float(backtest_results['sharpe'])
    }
}

# Save JSON summary
with open('alpha_discoveries.json', 'w') as f:
    json.dump(discoveries, f, indent=2)
print("✓ Saved: alpha_discoveries.json")

# Save models (for production use)
models = {
    'kmeans': finder.kmeans,
    'q_table': agent.q_table,
    'state_bins': agent.state_bins,
    'scaler': env.scaler,
    'state_features': env.state_features
}
with open('alpha_models.pkl', 'wb') as f:
    pickle.dump(models, f)
print("✓ Saved: alpha_models.pkl")

print("\n" + "=" * 70)
print("🚀 NEXT STEPS TO FIND BIGGER GAINS:")
print("=" * 70)
print("""
1. INCREASE TRAINING INTENSITY:
   - Change pop_size=500, generations=100 in Formula Evolution
   - Train RL agent for 2000+ episodes
   
2. TRY DIFFERENT ASSETS:
   - QQQ (tech-heavy), IWM (small caps), GLD (gold)
   - Individual stocks with high volume

3. USE SHORTER PREDICTION HORIZONS:
   - Change Future_5d_Return to Future_1d_Return or Future_3d_Return
   - This matches your 4.73% daily gain observation

4. ADD MORE FEATURES:
   - VIX (fear index)
   - Sector rotation indicators
   - Market breadth (advance/decline)
   - Order flow / dark pool data

5. MULTI-TIMEFRAME:
   - Run discovery on 1-hour, 4-hour, and daily simultaneously
   - Combine signals for confirmation

6. PRODUCTION DEPLOYMENT:
   - Load alpha_models.pkl in your trading system
   - Generate signals in real-time
   - Use ensemble.get_ensemble_signal(current_row)
""")
print("=" * 70)